In [3]:
from youtube_transcript_api import YouTubeTranscriptApi







In [4]:
def extract_video_id(video_id_or_url):
    if len(video_id_or_url) > 11:
        return video_id_or_url[-11:]
    else:
        return video_id_or_url

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(video_url_or_id):
    try:
        video_id = extract_video_id(video_url_or_id)
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"Error: {e}")
        return None

In [6]:

video_url = 'https://www.youtube.com/watch?v=yykGW30zgyE'
transcript = get_transcript(video_url)
video_info, video_title = my_utils.get_yt_video_info(video_url)


formatter = TextFormatter()

txt = formatter.format_transcript(transcript).replace("\n", " ")
txt


"if you want to see a real life example of how a marketing agency productized their service look no further in this episode I'm interviewing Spencer Powell of builderfunnel.com they are a marketing agency for custom home builders and remodelers in this episode he's actually going to share how he followed a lot of what we preach around having three income streams done for you a done with you in a DIY to transform his custom Services Agency into a very specialized productized delivery structure for his clients and how it's working extremely well so let's hop over into the interview alright guys so I'm joined here with Spencer Powell from Builder funnel and I'm super stoked to chat with him today and kind of have him share his story for the kickoff like Spencer is not a client we have not worked with him but he reached out after watching one of our YouTube videos around how agencies should have three income streams and we started chatting and he was sharing about everything he was doing a

In [7]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema.output_parser import StrOutputParser
from typing import List

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [8]:
model = ChatOpenAI()
prompt1 = ChatPromptTemplate.from_template(
"""
You extract surprising, insightful, and interesting information from text content. You are interested in insights related to the purpose and meaning of life, human flourishing, the role of technology in the future of humanity, artificial intelligence and its affect on humans, memes, learning, reading, books, continuous improvement, and similar topics.

Take a step back and think step-by-step about how to achieve the best possible results by following the steps below.

STEPS
1. Extract 5 to 15 of the most important, insightful, and/or interesting answered question in the input in a section called questions. If there are less than 15 then collect all of them. Make sure you extract at least 3.

OUTPUT INSTRUCTIONS
Only output Markdown.
Extract at least 5 questions answered from the content.
Do not give warnings or notes; only output the requested sections.
You use bulleted lists for output, not numbered lists.
Do not repeat questions.
Ensure you follow ALL these instructions when creating your output.

Input:
{transcript}
"""
)

model_parser = model | StrOutputParser()

chain = prompt1 | model | StrOutputParser()

chain.invoke({"transcript": txt})


from IPython.display import display, Markdown
display(Markdown(chain.invoke({"transcript": txt})))


# Questions

- How did Spencer Powell from Builder Funnel transform his custom services agency into a specialized productized delivery structure for his clients?
- What market selection process led Spencer to focus on serving remodeling companies and custom builders?
- Why did Spencer find it challenging to specialize and focus on a specific industry within the builder and remodeler space?
- How did Spencer make the decision to focus on remodelers and builders and what were the challenges he faced?
- How did Spencer's background in working with his family's home building and remodeling business influence his decision to specialize in serving remodelers and custom builders?
- What was the process behind Spencer's decision to create a blueprint offering for his clients and how did it transform his agency's delivery structure?
- How did Spencer navigate the transition from offering full-service retainer packages to implementing the blueprint and done with you productized services?
- What percentage breakdown does Spencer currently have for clients choosing the done for you, done with you, and DIY options in his agency?
- How did Spencer incorporate various marketing strategies like blogging, podcasts, YouTube, and trade shows to attract clients to his agency?
- What tools and strategies did Spencer use to manage the delivery of the blueprint and done with you services for his clients, ensuring ease of implementation and success?

In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Dict

class Questions(BaseModel):
    questions: Dict[str, str] = Field(
        ..., 
        description="Dynamic dictionary to store an arbitrary number of questions."
    )


In [10]:
parser = JsonOutputParser(pydantic_object=Questions)
prompt1 = ChatPromptTemplate.from_template(
    template = """
You extract surprising, insightful, and interesting information from text content. You are interested in insights related to the purpose and meaning of life, human flourishing, the role of technology in the future of humanity, artificial intelligence and its affect on humans, memes, learning, reading, books, continuous improvement, and similar topics.

Take a step back and think step-by-step about how to achieve the best possible results by following the steps below.

STEPS
1. Extract up to 50 of the most important, insightful, and/or interesting answered question in the input in a section called questions. If there are less than 15 then collect all of them. Make sure you extract at least 3.

OUTPUT INSTRUCTIONS
{format_instructions}

Input:
{transcript}
""",
    partial_variables={"format_instructions": parser.get_format_instructions()},

)


model_parser = model | parser

chain = prompt1 | model | parser

output = chain.invoke({"transcript": txt})



In [11]:
output

{'questions': {'1': 'Where did the market selection for specializing in remodelers come from?',
  '2': 'How did Spencer navigate the transition into making marketing easier for clients to implement?',
  '3': 'What is the breakdown of clients taking the done for you, done with you, and DIY options at Builder Funnel?',
  '4': 'What are some of the marketing strategies used by Builder Funnel to attract clients?',
  '5': 'How did the Blueprint offering impact the marketing and delivery of services at Builder Funnel?'}}

In [12]:
output
import json
json_string = json.dumps(output)

json_object = json.loads(json_string)

# Now you can access the data as a Python dictionary
# For example, to print all questions:
for key, question in json_object['questions'].items():
    print(f"{question}")


Where did the market selection for specializing in remodelers come from?
How did Spencer navigate the transition into making marketing easier for clients to implement?
What is the breakdown of clients taking the done for you, done with you, and DIY options at Builder Funnel?
What are some of the marketing strategies used by Builder Funnel to attract clients?
How did the Blueprint offering impact the marketing and delivery of services at Builder Funnel?


In [13]:
outputs_list = []
for question_number, question_text in output['questions'].items():
    print(f"{question_text}")
    prompt2 = ChatPromptTemplate.from_template(
    """
    Read the provided transcript and answer the provided question based on the information in the transcript. 

    Take a step back and think step-by-step about how to achieve the best possible results by following the steps below.

    INPUT
    {transcript}

    Question 
    {question}
    """
    )

    chain2 = prompt2 | model | StrOutputParser()

    output2 = chain2.invoke({"transcript": txt, "question": question_text})
    video_title = my_utils.sanitize_title_for_display(video_title)
    
    # Save question and output as a pair in outputs_list
    outputs_list.append((question_text, output2))
    
my_utils.save_to_unprocessed(outputs_list, video_title)

Where did the market selection for specializing in remodelers come from?
How did Spencer navigate the transition into making marketing easier for clients to implement?
What is the breakdown of clients taking the done for you, done with you, and DIY options at Builder Funnel?
What are some of the marketing strategies used by Builder Funnel to attract clients?
How did the Blueprint offering impact the marketing and delivery of services at Builder Funnel?


In [101]:
import sys
sys.path.append('C:\\Users\\vital\\iCloudDrive\\streamlit_suite\\src')
from my_utils.my_utils import MyUtils

my_utils = MyUtils()

my_utils.save_to_unprocessed(output2, )

In [2]:
from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

In [4]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://thedankoe.com/letters/zero-to-1-million-as-a-one-person-business-while-working-2-4-hours-per-day/"]
)
documents[0]


Document(id_='https://thedankoe.com/letters/zero-to-1-million-as-a-one-person-business-while-working-2-4-hours-per-day/', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[ ![Dan Koe\nLogo](data:image/svg+xml,%3Csvg%20xmlns=\'http://www.w3.org/2000/svg\'%20viewBox=\'0%200%20401%20311\'%3E%3C/svg%3E)\n![Dan Koe Logo](https://thedankoe.com/wp-content/uploads/2022/04/logo-\nwhite.png) ](https://thedankoe.com)\n\n  * [Letters](https://thedankoe.com/letters/)\n  * [Resources](/#resources)\n  * [Work With Me](https://intake.kortex.co)\n  * [About](/#about)\n\n____ Menu\n\n  * [Letters](https://thedankoe.com/letters/)\n  * [Resources](/#resources)\n  * [Work With Me](https://intake.kortex.co)\n  * [About](/#about)\n\n__ Search\n\nSearch\n\n__ Close this search box.\n\nNot A Subscriber?\n\n**Join 140,000+ getting mindf*cked every Saturday morning** while reading The\nKoe Letter (you\'ll learn a bit about life & business too.)\

In [69]:
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.file import FlatReader
from pathlib import Path
parser = MarkdownNodeParser()
md_docs = FlatReader().load_data(Path("C:\\Users\\vital\\iCloudDrive\\streamlit_suite\\src\\zero-to-1-million-as-a-one-person-business-while-working-2-4-hours-per-day.md"))
nodes = parser.get_nodes_from_documents(md_docs)



In [68]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

splitter = MarkdownHeaderTextSplitter([
    ("# ", "h1"),
    ("## ", "h2"),
    ("### ", "h3"),
    ("#### ", "h4"),
    ("##### ", "h5"),
    ("###### ", "h6"),
])

parser = MarkdownNodeParser()
md_docs = FlatReader().load_data(Path("C:\\Users\\vital\\iCloudDrive\\streamlit_suite\\src\\zero-to-1-million-as-a-one-person-business-while-working-2-4-hours-per-day.md"))
with open("C:\\Users\\vital\\iCloudDrive\\streamlit_suite\\src\\zero-to-1-million-as-a-one-person-business-while-working-2-4-hours-per-day.md", "r", encoding="utf-8") as file:
    md_docs = file.read()

md_docs_split = splitter.split_text(md_docs)
index = SummaryIndex.from_documents(md_docs_split)
query_engine = index.as_query_engine()
response = query_engine.query("Write a summary on the future of work.")
print(response)



AttributeError: 'Document' object has no attribute 'get_doc_id'

In [42]:
#https://docs.llamaindex.ai/en/stable/understanding/querying/querying.html
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

index = VectorStoreIndex.from_documents(md_docs)

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

response = query_engine.query("Write a summary on the future of work.")
display(Markdown(f"{response}"))


The future of work is envisioned as a transition towards creative pursuits within a technologically advanced society. This shift emphasizes personal growth, problem-solving, and embracing creativity as essential elements for meaningful and fulfilling endeavors. Education businesses are emerging as decentralized alternatives to traditional schooling, offering accessible and cost-effective learning opportunities. Overcoming imposter syndrome through honesty and focusing on helping others is highlighted as a key to success in the evolving work landscape. Additionally, the concept of Koe's Law underscores the evolution of work towards earning more in less time, promoting creativity, skill acquisition, and continuous improvement. Building an audience, transitioning from client work to group coaching, and productizing services are identified as crucial steps in scaling one-person businesses to reach significant financial milestones.

In [2]:
import os
import openai
import datetime
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
from trulens_eval import Tru
from llama_index.llms.openai import OpenAI
import numpy as np
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

from trulens_eval.feedback import Groundedness
from dotenv import load_dotenv, find_dotenv

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

from llama_index.readers.obsidian import ObsidianReader
from llama_index.core import VectorStoreIndex
import logging
import sys
import os
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage
from llama_index.llms.openai import OpenAI as LIOpenAI

from llama_index.core  import Document
document = md_docs
def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def build_sentence_window_index_nodes(
    nodes,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex(
            nodes, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine




def get_prebuilt_trulens_recorder(query_engine, app_id):
    openai = OpenAI()

    qa_relevance = (
        Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
        .on_input_output()
    )

    qs_relevance = (
        Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
        .on_input()
        .on(TruLlama.select_source_nodes().node.text)
        .aggregate(np.mean)
    )

    grounded = Groundedness(groundedness_provider=openai)

    groundedness = (
        Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
            .on(TruLlama.select_source_nodes().node.text)
            .on_output()
            .aggregate(grounded.grounded_statements_aggregator)
    )

    feedbacks = [qa_relevance, qs_relevance, groundedness]
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
    )
    return tru_recorder



def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

NameError: name 'md_docs' is not defined

In [59]:
llm = ChatOpenAI()
index = build_sentence_window_index(
    md_docs,
    llm=llm,
    save_dir="./sentence_index",
    )

eval_questions= [
    "How does the author justify the claim that starting a one-person business with minimal startup costs is feasible through solving a personal problem and creating an education business?",
    "What evidence does the article provide to support the profitability of selling information products based on personal achievements in areas like fitness, focus, or skills?",
    "In what ways does the author argue that education businesses represent the future of schooling and offer a decentralized alternative to traditional education models?",
    "How does the author address overcoming imposter syndrome, and what strategies are suggested for individuals to be honest about their journey and expertise?",
    "What arguments are made regarding the evolution of work towards more creative endeavors in a tech-enhanced society, and how is Koe's Law used to illustrate this shift?",
    "Can you identify the steps recommended for building an audience and transitioning from client work to group coaching to increase earning potential?",
    "How does productizing services and creating digital products contribute to income diversification and scalability, according to the article?",
    "What role does rapid iteration and detachment from manual labor play in scaling a one-person business to achieve $1 million+ per year, as per the author's viewpoint?",
    "What potential biases or weaknesses in the author's argumentation could affect the generalizability of the advice provided to a broader audience?",
    "Considering the strengths and evidence presented, how compelling is the article's roadmap for aspiring entrepreneurs and freelancers looking to grow their one-person business?"
  ]


#Tru().reset_database()
    
    
sentence_index_1 = build_sentence_window_index(
    md_docs,
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=1,
    save_dir="sentence_index_1-koe-v2",
)
sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index_1
)

tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='sentence window engine 1-koe-v2'
)
run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)

    
    
sentence_index_3 = build_sentence_window_index(
    md_docs,
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index_3-koe-v2",
)
sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)

tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3-koe-v2'
)
run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)
#Tru().run_dashboard()

sentence_index_10 = build_sentence_window_index(
    md_docs,
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=10,
    save_dir="sentence_index_10-koe-v2",
)
sentence_window_engine_10 = get_sentence_window_query_engine(
    sentence_index_10
)

tru_recorder_10 = get_prebuilt_trulens_recorder(
    sentence_window_engine_10,
    app_id='sentence window engine 10-koe-v2'
)
run_evals(eval_questions, tru_recorder_10, sentence_window_engine_10)


C:\Users\vital\AppData\Local\Temp\ipykernel_14332\2445352080.py:51: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


C:\Users\vital\AppData\Local\Temp\ipykernel_14332\2445352080.py:51: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


C:\Users\vital\AppData\Local\Temp\ipykernel_14332\2445352080.py:51: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [55]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

index = VectorStoreIndex.from_documents(md_docs)

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

response = query_engine.query("Write a summary on the future of work.")
display(Markdown(f"{response}"))


tru_recorder_VectorIndex = get_prebuilt_trulens_recorder(
    query_engine,
    app_id='VectorIndex_engine_topk10_simcutoff0.7'
)
run_evals(eval_questions, tru_recorder_VectorIndex, query_engine)

The future of work is envisioned as a transition towards creative pursuits within a technologically advanced society. This shift emphasizes personal growth, problem-solving, and embracing creativity as essential elements. Education businesses are seen as a decentralized alternative to traditional schooling, offering accessible and cost-effective learning opportunities. Overcoming imposter syndrome through honesty and focusing on helping others is highlighted as a key to success in the evolving work landscape. Koe's Law underscores the evolution of work towards earning more in less time, promoting creativity, skill acquisition, and continuous improvement. Strategies such as building an audience, transitioning to group coaching, and productizing services are emphasized as crucial steps in scaling one-person businesses for significant financial success.

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [57]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

index = VectorStoreIndex.from_documents(md_docs)

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)

response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

response = query_engine.query("Write a summary on the future of work.")
display(Markdown(f"{response}"))


tru_recorder_VectorIndex = get_prebuilt_trulens_recorder(
    query_engine,
    app_id='VectorIndex_engine_topk5_simcutoff0.7'
)
run_evals(eval_questions, tru_recorder_VectorIndex, query_engine)

The future of work is envisioned as a transition towards creative pursuits within a technologically advanced society. This shift emphasizes personal growth, problem-solving, and embracing creativity as essential elements for meaningful and fulfilling endeavors. Education businesses are emerging as decentralized alternatives to traditional schooling, providing accessible and cost-effective learning opportunities. Overcoming imposter syndrome through honesty and focusing on helping others is highlighted as a pathway to success in the creator economy. Additionally, the concept of Koe's Law underscores the evolution of work towards earning more in less time, promoting creativity, skill acquisition, and continuous improvement. Strategies such as building an audience, transitioning from client work to group coaching, and productizing services are crucial steps in scaling one-person businesses towards achieving significant financial success.

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [46]:
Tru().run_dashboard()


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.178.21:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [36]:
from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from IPython.display import Markdown, display

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    md_docs,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
)


query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)
response = query_engine.query(
    "Write a summary on the future of work.",
)
display(Markdown(f"{response}"))


KeyboardInterrupt: 

In [ ]:
article = documents[0].get_content()

In [ ]:


prompt = ChatPromptTemplate.from_template("""
You extract surprising, insightful, and interesting information from text content. Identify the Main Ideas Of the Article.

Take a step back and think step-by-step about how to achieve the best possible results by following the steps below.

STEPS:
1. Extract up to 15-50 of the most important, insightful, and/or interesting key points in a section called KEYPOINTS. If there are less than 15 then collect all of them. Make sure you extract at least 15.
2. Extract up to 5-30 of the most important, insightful, and/or supporting arguments in a section called ARGUMENTS. If there are less than 5 then collect all of them. Make sure you extract at least 5.
3. Extract up to 5-20 of the most important and/or insightful conclusions in a section called CONCLUSIONS. If there are less than 5 then collect all of them. Make sure you extract at least 5.

OUTPUT INSTRUCTIONS
- Only output Markdown.
- Extract at least 15 KEYPOINTS answered from the content as Markdown H1 headers.
- Extract at least 5 ARGUMENTS answered from the content as Markdown H1 headers.
- Extract at least 5 KEYPOINTS answered from the content as Markdown H1 headers.
- Do not give warnings or notes; only output the requested sections.
- You use bulleted lists for output, not numbered lists.
- Do not repeat key points, arguments or conclusions.
- Ensure you follow ALL these instructions when creating your output.

INPUT
{article}
"""
)
chain = prompt | model | StrOutputParser()

output = chain.invoke({"article": article})

output

# Nodes


In [3]:
Tru().reset_database()
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.file import FlatReader
from pathlib import Path
parser = MarkdownNodeParser()
md_docs = FlatReader().load_data(Path("C:\\Users\\vital\\iCloudDrive\\streamlit_suite\\src\\zero-to-1-million-as-a-one-person-business-while-working-2-4-hours-per-day.md"))
nodes = parser.get_nodes_from_documents(md_docs)


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [6]:
llm = ChatOpenAI()

eval_questions= [
    "How does the author justify the claim that starting a one-person business with minimal startup costs is feasible through solving a personal problem and creating an education business?",
    "What evidence does the article provide to support the profitability of selling information products based on personal achievements in areas like fitness, focus, or skills?",
    "In what ways does the author argue that education businesses represent the future of schooling and offer a decentralized alternative to traditional education models?",
    "How does the author address overcoming imposter syndrome, and what strategies are suggested for individuals to be honest about their journey and expertise?",
    "What arguments are made regarding the evolution of work towards more creative endeavors in a tech-enhanced society, and how is Koe's Law used to illustrate this shift?",
    "Can you identify the steps recommended for building an audience and transitioning from client work to group coaching to increase earning potential?",
    "How does productizing services and creating digital products contribute to income diversification and scalability, according to the article?",
    "What role does rapid iteration and detachment from manual labor play in scaling a one-person business to achieve $1 million+ per year, as per the author's viewpoint?",
    "What potential biases or weaknesses in the author's argumentation could affect the generalizability of the advice provided to a broader audience?",
    "Considering the strengths and evidence presented, how compelling is the article's roadmap for aspiring entrepreneurs and freelancers looking to grow their one-person business?"
  ]


#Tru().reset_database()
    
    
sentence_index_1 = build_sentence_window_index_nodes(
    nodes,
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=1,
    save_dir="sentence_index_1-koe-v3_node",
)
sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index_1
)

tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1,
    app_id='sentence window engine 1-koe-v3_node'
)
run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)

    
    
sentence_index_3 = build_sentence_window_index_nodes(
    nodes,
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index_3-koe-v3_node",
)
sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index_3
)

tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3-koe-v3_node'
)
run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)
#Tru().run_dashboard()

sentence_index_10 = build_sentence_window_index_nodes(
    nodes,
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=10,
    save_dir="sentence_index_10-koe-v3_node",
)
sentence_window_engine_10 = get_sentence_window_query_engine(
    sentence_index_10
)

tru_recorder_10 = get_prebuilt_trulens_recorder(
    sentence_window_engine_10,
    app_id='sentence window engine 10-koe-v3_node'
)
run_evals(eval_questions, tru_recorder_10, sentence_window_engine_10)


C:\Users\vital\AppData\Local\Temp\ipykernel_4268\133356209.py:83: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


C:\Users\vital\streamlit_suite\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
C:\Users\vital\AppData\Local\Temp\ipykernel_4268\133356209.py:83: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


C:\Users\vital\AppData\Local\Temp\ipykernel_4268\133356209.py:83: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [9]:
from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from IPython.display import Markdown, display
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

new_index_kg = KnowledgeGraphIndex(
    nodes,
    max_triplets_per_chunk=2,
    include_embeddings=True,
)
# query using top 3 triplets plus keywords (duplicate triplets are removed)
query_engine_kg = new_index_kg.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=10,
)
response = query_engine_kg.query(
    "Write a summary on the future of work.",
)
display(Markdown(f"{response}"))

tru_recorder_11 = get_prebuilt_trulens_recorder(
    query_engine_kg,
    app_id='knowledgegraph-koe-v3_node'
)
run_evals(eval_questions, tru_recorder_11, query_engine_kg)


The future of work is evolving towards creative endeavors in a tech-enhanced society, emphasizing personal growth, problem-solving, and embracing continuous improvement. Education businesses are emerging as decentralized alternatives to traditional schooling, offering accessible and cost-effective learning opportunities. Overcoming imposter syndrome through honesty and focusing on helping others can lead to successful ventures in the creator economy. The concept of evolving work to earn more in less time promotes creativity, skill acquisition, and problem-solving. Building an audience, transitioning from client work to group coaching, and productizing services are key steps in scaling a one-person business to reach significant financial success.

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


Validation error: 1 validation error for Rating
rating
  Value error, Rating must be between 0 and 10 [type=value_error, input_value=67, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error


In [14]:
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.file import FlatReader
from pathlib import Path
parser = MarkdownNodeParser()
md_docs = FlatReader().load_data(Path("C:\\Users\\vital\\iCloudDrive\\streamlit_suite\\src\\zero-to-1-million-as-a-one-person-business-while-working-2-4-hours-per-day.md"))
nodes = parser.get_nodes_from_documents(md_docs)



#https://docs.llamaindex.ai/en/stable/understanding/querying/querying.html
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

index10 = VectorStoreIndex(nodes)

retriever10 = VectorIndexRetriever(
    index=index10,
    similarity_top_k=10,
)

response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine_vectorindex10 = RetrieverQueryEngine(
    retriever=retriever10,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

response = query_engine_vectorindex10.query("Write a summary on the future of work.")
display(Markdown(f"{response}"))


tru_recorder_13 = get_prebuilt_trulens_recorder(
    query_engine_vectorindex10,
    app_id='vectorstore-koe-v3_node-sim10-cutoff0.7'
)
run_evals(eval_questions, tru_recorder_13, query_engine_vectorindex10)


Empty Response

In [12]:
index5 = VectorStoreIndex(nodes)

retriever5 = VectorIndexRetriever(
    index=index5,
    similarity_top_k=5,
)

response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine_vectorindex5 = RetrieverQueryEngine(
    retriever=retriever5,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

response = query_engine_vectorindex5.query("Write a summary on the future of work.")
display(Markdown(f"{response}"))


tru_recorder_14 = get_prebuilt_trulens_recorder(
    query_engine_vectorindex5,
    app_id='vectorstore-koe-v3_node-sim5-cutoff0.7'
)
run_evals(eval_questions, tru_recorder_14, query_engine_vectorindex5)

The future of work is evolving towards creative pursuits within a technology-driven society, emphasizing personal development and problem-solving skills. Education businesses are emerging as decentralized alternatives to traditional schooling, offering accessible and cost-effective learning opportunities. Overcoming imposter syndrome through honesty and focusing on assisting others can lead to success in the creator economy. Work is transitioning towards earning more in less time, promoting creativity, skill acquisition, and problem-solving. Key steps in scaling a one-person business to exceed $1 million annually include building an audience, transitioning from client work to group coaching, and productizing services.

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
